In [1]:
import psycopg2
import pandas as pd
import ast
import numpy as np
import os
from sqlalchemy import create_engine
from dotenv import load_dotenv


In [2]:

def get_db_cursor():
    username = "doadmin"
    password = 'yzmodwh2oh16iks6'
    host = 'db-postgresql-cl1-do-user-2276924-0.db.ondigitalocean.com'
    port = 25060
    database = 'MandatoryMetadata'
    schema = 'historical'

    conn = psycopg2.connect(host=host, database=database,
                            user=username, password=password, port=port)
    cur = conn.cursor()
    cur.execute("SET search_path TO " + schema)

    return conn, cur

In [3]:
conn, cur = get_db_cursor()


In [4]:
def convertSpectra(df):
    print("Converting spectra")
    df_ = pd.DataFrame([i[[i for i in i.keys()][0]] for i in df['averaged_spectra'].values],columns = np.arange(522,3977,2))
    df_.index = df.index
    print("Spectra converted")
    return df_

In [ ]:
df = pd.read_sql("""
        SELECT spectraldata.metadata_id, averaged_spectra, mandatorymetadata.sample_code  
        FROM spectraldata 
        INNER JOIN mandatorymetadata 
        ON mandatorymetadata.metadata_id = spectraldata.metadata_id 
        WHERE 
        is_finalized=True AND 
        passed=True AND 
        is_active=True AND 
        averaged=True AND 
        mandatorymetadata.sensor_id = 1 AND  
        mandatorymetadata.sample_type_id = 1 AND
        mandatorymetadata.sample_pretreatment_id = 1
        """,con=conn)


C:\Users\tsuma.thomas\AppData\Local\Temp\ipykernel_20276\2099194603.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("""


In [ ]:
import psycopg2
import pandas as pd
import ast
import numpy as np
import os
from dotenv import load_dotenv
from pathlib import Path
import glob
import pyodbc

def get_db_cursor():
    username = "doadmin"
    password = 'yzmodwh2oh16iks6'
    host = 'db-postgresql-cl1-do-user-2276924-0.db.ondigitalocean.com'
    port = 25060
    database = 'MandatoryMetadata'
    schema = 'historical'

    conn = psycopg2.connect(host=host, database=database,
                            user=username, password=password, port=port)
    cur = conn.cursor()
    cur.execute("SET search_path TO " + schema)

    return conn, cur
conn, cur = get_db_cursor()

def getSixMonthsData():
    query = f"""SELECT spectraldata.metadata_id, averaged_spectra, mandatorymetadata.sample_code  
        FROM spectraldata 
        INNER JOIN mandatorymetadata 
        ON mandatorymetadata.metadata_id = spectraldata.metadata_id 
        WHERE 
        is_finalized=True AND 
        passed=True AND 
        is_active=True AND 
        averaged=True AND 
        mandatorymetadata.sensor_id = 1 AND  
        mandatorymetadata.sample_type_id = 1 AND
        mandatorymetadata.sample_pretreatment_id = 1 AND
        mandatorymetadata.timestamp > 2024-09-01
    """
    spc = pd.read_sql(query, con=conn)
    conn.close()
    spc = spc[['sample_code', 'averaged_spectra']]
    spc = spc.set_index('sample_code')
    spc = convertSpectra(spc)
    # spc.to_csv('outputFiles/6monthsspc.csv')
    return spc

def convertSpectra(df):
    print("Converting spectra")
    df_ = pd.DataFrame([i[[i for i in i.keys()][0]] for i in df['averaged_spectra'].values],columns = np.arange(522,3977,2))
    df_.index = df.index
    print("Spectra converted")
    return df_

In [ ]:
spc = getSixMonthsData()

: 